# File Validation Script-(CIM to amperity) 

# Instructions

Inputs
--Table names of Source and Target tables,
--Source and target table column names,
--Number of primary keys,
--Output file location(Query file),
--Number of discrepancy samples required,
--Discrepancies file generated by executing the query

Outputs
--Text file consisting of Queries for:
    Lookup rate
    Match rate
    #N/A values
    Discrepancies

--Excel file consisting of the formatted discrepanicies 

NOTE1:Make sure that the Primary key columns are in the starting of the input (Source and target table column names)
NOTE2:Alias for source table is cim and for target is amperity

In [1]:
import tkinter as tk
from tkinter.filedialog import askopenfilename
from tkinter.filedialog import askdirectory
from datetime import date
import pandas as pd
import numpy as np

import openpyxl
from openpyxl import Workbook

In [2]:
def Output_directory():
    root = tk.Tk()
    directory = askdirectory(title = 'Choose a folder for saving the generated script file')
    print(directory)
    root.destroy()
    return directory

In [3]:
#Lookup

def lookup_query_fun(cim,amperity,source_table_name,target_table_name,source_col_new,target_col_new,rename_file_loc):
    
    cim_count = f"(SELECT COUNT(*) FROM {source_table_name}) as lookup_rate"
    
    lookup_rate=f'''

--Lookup Query
select count(*) /{cim_count}
from {source_table_name} cim 
inner join 
{target_table_name} amperity 
on''' 

    x=''
    for i in range (0,no_of_keys_in_pk):
        x+='\n'
        x+=f"rtrim(ltrim(cast({cim}{source_col_new[i]} as string),'0'),' ')=rtrim(ltrim(cast({amperity}{target_col_new[i]} as string),'0'),' ') and"
    lookup_rate+=x[:len(x)-4]+';'

    file_path=rename_file_loc
    with open(file_path,'a') as file:
        file.write(lookup_rate)
#     print(lookup_rate)

In [4]:
#Match rate

def match_query_fun(cim,amperity,source_table_name,target_table_name,source_col_new,target_col_new,rename_file_loc):
    match_count=f'''
    
--Match count Query
Select
    '''
    x=''
    #adding match count
    for i in range (len(source_col_new)):
        x+=f'''Count(Case when ifnull(rtrim(ltrim(upper(cast({cim}{source_col_new[i]} as string)),'0'),'0'),' ')= ifnull(rtrim(ltrim(upper(cast({amperity}{target_col_new[i]} as string)),'0'),'0'),' ') then ifnull(cast(cim.{source_col_new[i]} as string) ,'0') End) as count_{source_col_new[i]},'''
        x+='\n'
    x=x[:len(x)-2]+f'''
from 
{source_table_name} cim 
inner join 
{target_table_name} amperity 
on'''
    #adding join condition
    for i in range (0,no_of_keys_in_pk):
        x+='\n'
        x+=f"rtrim(ltrim(cast({cim}{source_col_new[i]} as string),'0'),' ')=rtrim(ltrim(cast({amperity}{target_col_new[i]} as string),'0'),' ') and"
    match_count+=x[:len(x)-4]+';'

    file_path=rename_file_loc
    with open(file_path,'a') as file:
        file.write(match_count)
    #print(match_count)

In [5]:
#NA Values

def na_values_query_fun(cim,amperity,source_table_name,target_table_name,source_col_new,target_col_new,rename_file_loc):
    na_values=f'''

--#N/A Values Query
select 
* 
from 
{source_table_name} cim
left outer join 
{target_table_name} amperity
on
'''
    x=''
    for i in range (0,no_of_keys_in_pk):
        x+=f"rtrim(ltrim(cast({cim}{source_col_new[i]} as string),'0'),' ')=rtrim(ltrim(cast({amperity}{target_col_new[i]} as string),'0'),' ') and"
        x+='\n'

    #join condition    
    y=x[:len(x)-5]+f'''
where
    '''
    z=''
    #where condition
    for i in range (0,no_of_keys_in_pk):    
        z+=f''' {amperity}{target_col_new[i]} is null and'''
        z+='\n'

    na_values+=y+z[:len(z)-5]+';'

    file_path=rename_file_loc
    with open(file_path,'a') as file:
        file.write(na_values)
    #print(na_values)

In [6]:
#Discrepancies
#Takes care of case-sensitivity 
#Takes care of trailing spaces and leading zeros

def discrepancies_query(source_col_new1,target_col_new1,source_table_name,target_table_name,cim,amperity,no_of_keys_in_pk,no_of_samples):
    columns=''
    for i in range(len(source_col_new1)):
        columns+='cim.'+source_col_new1[i]+' as '+source_col_new1[i]+'_cim ,'+'amperity.'+target_col_new1[i]+' as '+target_col_new1[i]+'_amperity ,'

# #select
#         discrepancies=f'''
        
# --Discrepancies Query
# (select {columns[:-1]},
# from {source_table_name} cim 
# inner join 
# {target_table_name} amperity 
# on''' 

#On condition        
    x=''
    for i in range (0,no_of_keys_in_pk):
        x+='\n'
        x+=f"rtrim(ltrim(cast({cim}{source_col_new[i]} as string),'0'),' ')=rtrim(ltrim(cast({amperity}{target_col_new[i]} as string),'0'),' ') and"

#where and select
    where_cond_and_disc=''
    for i in range(len(source_col_new1)):
        discrepancies=f'''
(select {columns[:-1]},'{source_col_new1[i]}' as column
from {source_table_name} cim 
inner join 
{target_table_name} amperity 
on'''
        where_cond_and_disc+=discrepancies+x[:len(x)-4]+ f''' 
where 
ifnull(rtrim(ltrim(upper(cast({cim}{source_col_new[i]} as string)),'0'),' '),'0') != ifnull(rtrim(ltrim(upper(cast({amperity}{target_col_new[i]} as string)),'0'),' '),'0') 
order by rand() 
limit {no_of_samples})'''+ ' UNION all'+'\n'
    
    disc_query=f'''
--Discrepancies Query
'''
    disc_query+='\n'+where_cond_and_disc[:-10]+"\n"+';'
    
    file_path=rename_file_loc
    
    with open(file_path,'a') as file:
        file.write(disc_query)
#     print(disc_query)

In [7]:
#Temp table creation by taking file name and gs location as input

source_table_input=input(f"Enter name of the source table (CIM)  \n")

target_table_input=input(f"Enter name of the target table (Amperity)  \n")

Enter name of the source table (CIM)  
cte
Enter name of the target table (Amperity)  
cte_1


In [8]:
#Taking input of table names,column names, column lengths and number of columns in primary key
no_of_keys_in_pk=int(input("No. of primary keys "))

#Output file name and Directory
op_suff= Output_directory()
rename_file_loc1=input("Enter the name of the script file ")
rename_file_loc = op_suff+'/'+'{i}.txt'.format(i = rename_file_loc1)

No. of primary keys 3
C:/Users/shivadatt.k/Desktop/Amperity/resp
Enter the name of the script file query1


In [9]:
#names of temp tables created 
source_table_name=f'''{source_table_input}'''
target_table_name=f'''{target_table_input}'''
#print("\n",source_table_name,"\n\n",target_table_name)

In [13]:
print("Enter the column names as a comma(,) seperated string")
source_col_new=(input("\nEnter the new source column name (Ex:'indiv_id,hh_id,fin_divn_nbr....'): \n").split(','))
target_col_new=(input("\nEnter the new target column name (Ex:'indiv_id,hh_id,fin_divn_nbr....'): \n").split(','))
source_col_new1,target_col_new1,cim,amperity=source_col_new,target_col_new,'cim.','amperity.'

Enter the column names as a comma(,) seperated string

Enter the new source column name (Ex:'indiv_id,hh_id,fin_divn_nbr....'): 
email_address,treatment_code,campcell_key,ss_key,date_key,transtype_key,indiv_key,dim_cme_key,email_history_key,time_key,personalization_key,email_type_cme_key,occur,create_date,daas_crt_ts,daas_upd_ts,daas_load_id,daas_load_user_id,dt,content_details,banner_recs,offer_recs,dynamic_content_signature_id,rn

Enter the new target column name (Ex:'indiv_id,hh_id,fin_divn_nbr....'): 
email_address,treatment_code,campcell_key,ss_key,date_key,transtype_key,indiv_key,dim_cme_key,email_history_key,time_key,personalization_key,email_type_cme_key,occur,create_date,daas_crt_ts,daas_upd_ts,daas_load_id,daas_load_user_id,dt,content_details,banner_recs,offer_recs,dynamic_content_signature_id,rn


In [14]:
#Discrepencies are needed for which columns?
#Do you want All columns or Do you want to select the columns
#Please keep the primary key columns in the starting

def check_all_columns_for_discrepancies():
    response=input("Do you want discrepancies for all the columns --Yes or No?")
    if response.upper()=="NO":
        source_col_new1=input("Enter the source column names for which discrepancies are requires seperated by commas").split(",")
        #target_col_new1=input("Enter the target column names for which discrepancies are requires seperated by commas").split(",")
        target_col_new1=source_col_new1
        return source_col_new1,target_col_new1


In [15]:
#Running all the functions

lookup_query_fun(cim,amperity,source_table_name,target_table_name,source_col_new1,target_col_new1,rename_file_loc)
x=match_query_fun(cim,amperity,source_table_name,target_table_name,source_col_new1,target_col_new1,rename_file_loc)
na_values_query_fun(cim,amperity,source_table_name,target_table_name,source_col_new1,target_col_new1,rename_file_loc)

no_of_samples=int(input("Enter the no. of discepancy samples required for each column"))
check_all_columns_for_discrepancies()
discrepancies_query(source_col_new1,target_col_new1,source_table_name,target_table_name,cim,amperity,no_of_keys_in_pk,no_of_samples)

print("The sql scipt file is generated")

Enter the no. of discepancy samples required for each column5
Do you want discrepancies for all the columns --Yes or No?yes
The sql scipt file is generated


In [ ]:
#discrepancies sorting

#Step 1-Take discrepancies sheet as input
root = tk.Tk()
filepath = askopenfilename(title = 'Select the discrepancies file downloaded from GCP')
root.destroy()

pd_discrepancies=pd.read_csv(filepath)
pd_discrepancies
column_disc=list(pd_discrepancies['column'].unique())
no_of_disc_cols=len(column_disc)

#Step 2-Find the column 
list_disc=[]
pk=[]
for i in range(0,no_of_keys_in_pk*2):
    pk.append(pd_discrepancies.columns[i])
for i in range(0,no_of_disc_cols):
    col_cim=column_disc[i]+'_cim'
    col_amperity=column_disc[i]+'_amperity'
#     print(col_amperity)
    df=pd.DataFrame(columns=pk+[col_cim,col_amperity])
    for j in range(len(pd_discrepancies)):
        x=[]
        if pd_discrepancies['column'].iloc[j]==column_disc[i]:
            if pd_discrepancies[column_disc[i]+'_cim'].iloc[j]!=pd_discrepancies[column_disc[i]+'_amperity'].iloc[j]:
                x = [
    pd_discrepancies[element].iloc[j] for element in pk
] + [
    pd_discrepancies[column_disc[i] + '_cim'].iloc[j],
    pd_discrepancies[column_disc[i] + '_amperity'].iloc[j]
]
                df.loc[len(df)]=x
    if len(df)>0:
        list_disc.append(df)
        
# Step 3-Formating the output
output_file =op_suff+'/'+'discrepancies'+source_table_name+'.xlsx'
excel_writer = pd.ExcelWriter(output_file, engine='openpyxl')

start_row = 1
for df in list_disc:
    df.to_excel(excel_writer, sheet_name='Sheet1', startrow=start_row, startcol=0, index=False)
    start_row += df.shape[0] + 2 
    
# Save and close the Excel file
excel_writer.save()
excel_writer.close()

print(f'DataFrames written to {output_file}')

In [ ]:
discrepancies_query


In [ ]:
A=['a','b','c']
b=[i for i in A].add('x')


In [ ]:
b